<a href="https://colab.research.google.com/github/ahmed-tbe/bilayer-graphene/blob/main/graphene_periodicity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

# === USER CONFIGURATION ===
input_file = "graphene_trimmed.data"
output_file = "graphene_trimmed_manual_periodic.data"

# Lattice constants
a_x = 2.46  # zigzag direction (x)
a_y = 4.26  # armchair direction (y)

# Manual trimming: number of unit cells to trim from each side
trim_left = 1     # in units of a_x
trim_right = 1
trim_bottom = 1   # in units of a_y
trim_top = 1

# Tolerance for matching z-layers
z_top = 3.35
z_bot = 0.00
z_tol = 0.2

# === LOAD FILE ===
with open(input_file, 'r') as f:
    lines = f.readlines()

start_idx = next(i for i, line in enumerate(lines) if line.strip() == "Atoms") + 2
header_lines = lines[:start_idx]
atom_lines = lines[start_idx:]

atoms = []
for line in atom_lines:
    if not line.strip():
        continue
    parts = line.strip().split()
    atom_id = int(parts[0])
    atom_type = int(parts[1])
    charge = float(parts[2])
    x, y, z = map(float, parts[3:6])
    atoms.append((atom_id, atom_type, charge, x, y, z))

# === SPLIT LAYERS ===
top_layer = [a for a in atoms if abs(a[5] - z_top) < z_tol]
bottom_layer = [a for a in atoms if abs(a[5] - z_bot) < z_tol]

# === COMBINE FOR GLOBAL SHIFT ===
combined = top_layer + bottom_layer
x_min = min(a[3] for a in combined)
y_min = min(a[4] for a in combined)

# Shift to origin
shifted_atoms = []
for (i, t, c, x, y, z) in combined:
    shifted_atoms.append((i, t, c, x - x_min, y - y_min, z))

# === TRIM BASED ON UNIT CELLS ===
x_coords = [a[3] for a in shifted_atoms]
y_coords = [a[4] for a in shifted_atoms]
x_len = max(x_coords)
y_len = max(y_coords)
N_x = int(round(x_len / a_x))
N_y = int(round(y_len / a_y))

x_lo = trim_left * a_x
x_hi = (N_x - trim_right) * a_x
y_lo = trim_bottom * a_y
y_hi = (N_y - trim_top) * a_y

# Final trimmed region
final_atoms = [
    a for a in shifted_atoms
    if (x_lo - 1e-4) <= a[3] <= (x_hi + 1e-4) and (y_lo - 1e-4) <= a[4] <= (y_hi + 1e-4)
]

# === REINDEX ===
reindexed = [
    (i + 1, a[1], a[2], a[3] - x_lo, a[4] - y_lo, a[5]) for i, a in enumerate(final_atoms)
]

# === UPDATE HEADER ===
new_xhi = x_hi - x_lo
new_yhi = y_hi - y_lo
z_min = min(a[5] for a in reindexed)
z_max = max(a[5] for a in reindexed)

updated_header = []
for line in header_lines:
    if "xlo xhi" in line:
        updated_header.append(f"0.000000 {new_xhi:.6f} xlo xhi\n")
    elif "ylo yhi" in line:
        updated_header.append(f"0.000000 {new_yhi:.6f} ylo yhi\n")
    elif "zlo zhi" in line:
        updated_header.append(f"{z_min:.6f} {z_max:.6f} zlo zhi\n")
    elif "atoms" in line:
        updated_header.append(re.sub(r'^\d+', str(len(reindexed)), line))
    else:
        updated_header.append(line)

# === WRITE OUTPUT ===
with open(output_file, 'w') as f:
    f.writelines(updated_header)
    f.write("\nAtoms\n\n")
    for a in reindexed:
        f.write(f"{a[0]} {a[1]} {a[2]:.6f} {a[3]:.6f} {a[4]:.6f} {a[5]:.6f}\n")

print(f"✅ Saved: {output_file}")
print(f"📏 Box: x = {new_xhi:.2f} Å, y = {new_yhi:.2f} Å, z = {z_max - z_min:.2f} Å")


✅ Saved: graphene_trimmed_manual_periodic.data
📏 Box: x = 93.48 Å, y = 93.72 Å, z = 3.35 Å
